# Creating the Enviornment:

In [1]:
!pip install streamlit langchain langchain_huggingface langchain_chroma langchain_text_splitters transformers sentence-transformers chromadb pyngrok


INFO: pip is looking at multiple versions of langchain-huggingface to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langchain-chroma to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 76.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 68.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 63.7 MB/s eta 0:00:00

In [1]:
!pip install langchain langchain_community langchain_chroma

INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 449.8/449.8 kB 23.9 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.0.3
    Uninstalling langchain-core-1.0.3:
      Successfully uninstalled langchain-core-1.0.3
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 1.0.0
    Uninstalling langchain-text-splitters-1.0.0:
      Successfully uninstalled langchain-text-splitters-1.0.0
  Attempting uninstall: langchain_community
    Found existing installation: langchain-community 0.4.1
    Uninstalling langchain-community-0.4.1:
      Successfully uninstalled langchain-community-0.4.1
ERROR: pip's dependency resolver does not currently take into account all 

# Importing the Libraries:

In [2]:
import warnings
warnings.filterwarnings("ignore")

from langchain_chroma import Chroma # Virtual DB
from langchain_core.prompts import PromptTemplate #Custom template of prompting
from langchain_text_splitters import CharacterTextSplitter # Character --> Chunks
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers.string import StrOutputParser #Providing output
from langchain_huggingface import HuggingFaceEmbeddings #Embedding model importing
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline  # For creating model pipeline
from langchain.llms import HuggingFacePipeline # Corrected import
import asyncio, os

# Uploading and reading the file:

In [3]:
with open('/content/rag text.txt') as f:
  file=f.read()

# Creating Chunks:

In [4]:
text_split=CharacterTextSplitter(chunk_size=1000,chunk_overlap=200,length_function=len)
chunks=text_split.create_documents([file])
print(f"Created {len(chunks)} chunks.")

Created 2 chunks.


In [5]:
chunks[0]

Document(metadata={}, page_content='🏫 Academic Journey:\nBorn and raised in New Barrackpore, I graduated with distinction from New Barrackpore Colony Boys’ High School, securing 85% in both Secondary and Higher Secondary. Currently pursuing B.Tech in Computer Science & Engineering (AI & ML) from Techno Engineering College, Banipur.\n\n📊 Professional Passion:\nI’m currently enrolled in an industry-aligned AI & Data Science program by IIT Indore in collaboration with Intellipaat. My main interest lies in data analytics — deriving stories from numbers, building insights from raw information, and applying them in impactful ways.\n\n💪 Personal Drive:\nWith 4+ years of dedication to fitness and gym training, I believe in goal-setting, consistency, and personal discipline. My fitness journey reflects the same mindset I apply to analytics — steady growth, smart decisions, and resilience.')

In [6]:
chunks[1]

Document(metadata={}, page_content='🚀 Looking Forward:\nI’m eager to connect with professionals, recruiters, and mentors who can guide or provide opportunities in data analytics, AI/ML, or business intelligence. Let’s connect!')

# Creating Embeddings and VectorDB:

In [9]:
embedding_model = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')
vectorDB = Chroma(collection_name="suryavhi", embedding_function=embedding_model)

await vectorDB.aadd_documents(chunks)
retriever = vectorDB.as_retriever()
print("✅ Chroma Vector DB ready.")

✅ Chroma Vector DB ready.


# Building the LLM:

In [10]:
tokenizer = AutoTokenizer.from_pretrained('google/flan-t5-base')

if tokenizer.pad_token is None:
  tokenizer.add_special_tokens({'pad_toke':'[PAD]'})

model=AutoModelForSeq2SeqLM.from_pretrained('google/flan-t5-base')
model.resize_token_embeddings(len(tokenizer))
model.config.pad_token_id = tokenizer.pad_token_id

generator=pipeline('text2text-generation',model=model,tokenizer=tokenizer,max_new_tokens=150)
LLM = HuggingFacePipeline(pipeline=generator)

print("✅ Model loaded.")

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Device set to use cpu


✅ Model loaded.


/tmp/ipython-input-51225510.py:11: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  LLM = HuggingFacePipeline(pipeline=generator)


# Prompt Template Generation:

In [11]:
template = """Use the context provided to answer the question.
If you don't know, say "I don't know."

Context:
{context}

Question:
{question}

Answer:"""

prompt = PromptTemplate(template=template)

In [12]:
prompt

PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Use the context provided to answer the question.\nIf you don\'t know, say "I don\'t know."\n\nContext:\n{context}\n\nQuestion:\n{question}\n\nAnswer:')

# RAG chain Generation:

In [13]:
rag_chain=(
    {'context':retriever,'question':RunnablePassthrough()}
    | prompt
    | LLM
    | StrOutputParser()
)

print("✅ RAG chain built.")

✅ RAG chain built.


# Question and Answering:

In [14]:
question = "Where does Suryavhi das got interest?"
answer = rag_chain.invoke(question)
print("🔹 Question:", question)
print("🔹 Answer:", answer)

🔹 Question: Where does Suryavhi das got interest?
🔹 Answer: AI & Data Science program by IIT Indore in collaboration with Intellipaat


In [16]:
question = "What is the name of the student?"
answer = rag_chain.invoke(question)
print("🔹 Question:", question)
print("🔹 Answer:", answer)


🔹 Question: What is the name of the student?
🔹 Answer: I don't know.


# Gradio Interface:

In [19]:
!pip install Gradio

In [22]:
# Function for inference
import gradio as gr

def ask_question(user_question):
    if not user_question.strip():
        return "⚠️ Please enter a valid question."
    try:
        answer = rag_chain.invoke(user_question)
        return f"**Question:** {user_question}\n\n**Answer:** {answer}"
    except Exception as e:
        return f"❌ Error: {str(e)}"

# Define the UI
with gr.Blocks(theme=gr.themes.Soft()) as rag_app:
    gr.Markdown("""
    <h1 style='text-align:center; color:#2E86C1;'>🤖 RAG Chat Assistant</h1>
    <p style='text-align:center;'>Ask questions based on your uploaded knowledge base 📘</p>
    <hr>
    """)

    with gr.Row():
        with gr.Column(scale=6):
            question_box = gr.Textbox(
                label="💬 Ask a Question",
                placeholder="Type your question here...",
                lines=2
            )
            submit_btn = gr.Button("🚀 Generate Answer", variant="primary")
        with gr.Column(scale=6):
            output_box = gr.Markdown(label="🧠 Model Response")

    submit_btn.click(fn=ask_question, inputs=question_box, outputs=output_box)

    gr.Markdown("""
    <hr>
    <p style='text-align:center; color:gray;'>Built with ❤️ using LangChain + Gradio</p>
    """)

rag_app.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c3449c303caa8d28fa.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
